In [136]:
from collections import defaultdict
import numpy as np
import pandas as pd
from conllu import parse_incr, TokenList
from enum import Enum
from typing import Iterator, List, Dict, Tuple

In [172]:
class SmoothingStrategy(Enum):
    first = 1


smoothing_strategy = None
train = open("data/train.conllu", "r", encoding="utf-8")
test = open("data/test.conllu", "r", encoding="utf-8")
val = open("data/val.conllu", "r", encoding="utf-8")
tags = ['START', 'O', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'B-LOC', 'B-MISC', 'B-PER', 'B-ORG', 'END']


## Smoothing strategies

In [173]:
# TODO Add the remaining smoothing strategies
def unknown_word_emission(smoothing_strategy: Enum) -> float:
    if smoothing_strategy == SmoothingStrategy.first:
        return 1 / len(tags)
    return 0

## Matrix Functions

In [174]:
def get_transition_matrix(tags: List[str], train: Iterator[TokenList]) -> np.array:
    transition_matrix = np.zeros((len(tags), len(tags)), dtype=float)

    tag_counter = defaultdict(int)
    transition_counter = defaultdict(int)

    for sentence in parse_incr(train):
        # count first tag of sentence and match it with 'START' artificial tag
        first_tag = sentence[0]['lemma']
        tag_counter['START'] += 1
        transition_counter[('START', first_tag)] += 1

        # count middle token pairs
        for (token_a, token_b) in zip(sentence, sentence[1:]):
            tag_counter[token_a['lemma']] += 1
            transition_counter[(token_a['lemma'], token_b['lemma'])] += 1

        # count last tag of sentence and match it with 'END' artificial tag
        last_tag = sentence[-1]['lemma']
        tag_counter[last_tag] += 1
        transition_counter[(last_tag, 'END')] += 1

    for i, t1 in enumerate(tags):
        for j, t2 in enumerate(tags):
            if tag_counter[t1]:  # if tag occurs at least once
                transition_matrix[i][j] = transition_counter[(t1, t2)] / tag_counter[t1]  # compute transition probability

    train.seek(0)
    return transition_matrix


def get_emission_probabilities(train: Iterator[TokenList]) -> Dict[str, str]:
    word_tag_counter = defaultdict(int)
    tag_counter = defaultdict(int)

    for sentence in parse_incr(train):
        for token in sentence:
            word_tag_counter[(token['form'], token['lemma'])] += 1
            tag_counter[token['lemma']] += 1

    emission_probabilities = {(word, tag): count / tag_counter[tag] for (word, tag), count in word_tag_counter.items()}  # compute emission probability
    return emission_probabilities


def get_emission_matrix(tags: List[str], words: List[str], emission_probabilities: [Dict[str, str]]) -> np.array:
    emission_matrix = np.zeros((len(tags), len(words)), dtype='float32')
    for i, tag in enumerate(tags):
        for j, word in enumerate(words):
            emission_matrix[i, j] = emission_probabilities.get((word, tag), unknown_word_emission(smoothing_strategy))

    return emission_matrix


## Learning

In [175]:
transition_matrix = get_transition_matrix(tags, train)
transition_matrix = transition_matrix[:-1, 1:] # remove last row (END) and first column (START)

emission_probabilities = get_emission_probabilities(train)
train.close()

## Definition of Viterbi

In [148]:
# TODO Need to use logarithm to compute argmax probability in order to improve performance
def argmax(viterbi_matrix: np.array, Tm: np.array, Em: np.array, j: int, i: int) -> Tuple[int, float]:
    max_prob = 0
    max_index = 0
    for index, prob in enumerate(viterbi_matrix[:, i-1]):
        prob *= Tm[index+1, j] * Em[j, i]
        if prob > max_prob:
            max_index, max_prob = index, prob
    
    return max_index, max_prob


def viterbi(words: List[str], tags: List[str], Π: np.array, Tm: np.array, Em: np.array) -> List[str]:
    """
    Parameters
    ----------
    words: List[str]
        Words sequence of the sentence
    tags: List[str]
        List of NER tags
    Π: np.array
        Array of initial probabilities (1, T)
    Tm: np.array
        Transition matrix (T, T)
    Em: np.array
        Emission matrix (T, W)
    """

    W = len(words)
    T = len(tags)

    viterbi_matrix = np.zeros((T, W), dtype=float)
    backpointer = np.empty((T, W), dtype=int)

    # compute first word initial probability for each tag
    viterbi_matrix[:, 0] = [emission * initial_p for emission, initial_p in zip(Em[:, 0], Π)]

    # compute probabilities and fill backpointer for the rest of matrix
    for i in range(1, W):
        for j in range(T):
            k, value = argmax(viterbi_matrix, Tm, Em, j, i)
            viterbi_matrix[j, i] = value
            backpointer[j, i] = k

    # get tag index k of last column with highest probability
    last_column = list(viterbi_matrix[:, -1])
    max_value = max(last_column)
    k = last_column.index(max_value)

    # get best path walking through backpointer
    best_path = list()
    for i in range(W-1, -1, -1):
        best_path.append(tags[k])
        k = backpointer[k, i]
    
    best_path.reverse()
    return best_path

## Evaluation

In [176]:
tag_subset = tags[1:-1] # remove artificial tags START and END

total_words = 0
correctly_classified = 0
for sentence in parse_incr(test):
    total_words += len(sentence)
    correct_tags = [token['lemma'] for token in sentence]
    words = np.array([token['form'] for token in sentence])

    emission_matrix = get_emission_matrix(tag_subset, words, emission_probabilities)
    Π = transition_matrix[0, :-1]

    result_tags = viterbi(words, tag_subset, Π, transition_matrix, emission_matrix)
    correctly_classified += sum(x == y for x, y in zip(correct_tags, result_tags))

print(f'Total words: {total_words}')
print(f'Correctly tagged: {correctly_classified}')
print(f'Accuracy: {round(correctly_classified / total_words * 100, 2)}%')

Total words: 267155
Correctly tagged: 250487
Accuracy: 93.76%
